In [32]:
!pip3 install datasets
from datasets import load_dataset

ds = load_dataset("stanfordnlp/imdb")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip


In [33]:
# take a look at the first example
print(ds['test'][0])

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [35]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf 

model_name = "harshhmaniya/fine-tuned-bert-imdb-sentiment-analysis"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

custom_text = "I absolutely loved this movie! The plot was gripping and the acting was top-notch."
test_text = ds['test'][0]['text']

inputs = tokenizer(custom_text, return_tensors="tf", truncation=True, padding=True)

outputs = model(inputs)
logits = outputs.logits


probs = tf.nn.softmax(logits, axis=-1)
pred_class = tf.argmax(probs, axis=-1).numpy()[0]

label_map = {0: "Negative", 1: "Positive"}
print(f"Review Sentiment: {label_map[pred_class]}")

Some layers from the model checkpoint at harshhmaniya/fine-tuned-bert-imdb-sentiment-analysis were not used when initializing TFBertForSequenceClassification: ['dropout_113']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at harshhmaniya/fine-tuned-bert-imdb-sentiment-analysis.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


Review Sentiment: Positive


## Use Gen AI to generate a movie review

In [ ]:
# !pip3 install openai

import openai
import os

# Set API key (Replace with your actual key or use environment variables)
openai.api_key = "inset_your_api_key_here"

In [ ]:
def generate_movie_review(movie_title, genre, sentiment="positive"):
    prompt = f"Write a {sentiment} movie review for '{movie_title}', a {genre} film."

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a movie critic."},
                  {"role": "user", "content": prompt}],
        max_tokens=200  
    )

    return response["choices"][0]["message"]["content"]

movie_review = generate_movie_review("Inception", "sci-fi", "positive")
print(movie_review)